#### Thursday, April 4, 2024

https://microsoft.github.io/autogen/docs/tutorial/chat-termination

This all runs. Nice!

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/NexusRaven-V2-13B-GGUF/nexusraven-v2-13b.Q8_0.gguf",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I am a humble AI,\nMy purpose is to learn and grow,\nI'm always ready to play,\nBut I don't know the rules yet.", role='assistant', function_call=None, tool_calls=None)


In [2]:
config_list=[
    {
        "model": "LMStudio Local Model",
        "base_url": "http://localhost:1234/v1",
        "api_type": "open_ai",
        "api_key": "NULL", # required, you don´t need to change this
        }
]

llm_config={
    "config_list": config_list,
}

# Terminating Conversations Between Agents

In this chapter, we will explore how to terminate a conversation between AutoGen agents.

_But why is this important?_ Its because in any complex, autonomous workflows it's crucial to know when to stop the workflow. For example, when the task is completed, or perhaps when the process has consumed enough resources and needs to either stop or adopt different strategies, such as user intervention. So AutoGen natively supports several mechanisms to terminate conversations.

How to Control Termination with AutoGen?
Currently there are two broad mechanism to control the termination of conversations between agents:

1. **Specify parameters in `initiate_chat`**: When initiating a chat, you can define parameters that determine when the conversation should end.

2. **Configure an agent to trigger termination**: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.


## Parameters in `initiate_chat`
In the previous chapter we actually demonstrated this when we used the `max_turns` parameter to limit the number of turns. If we increase `max_turns` to say `3` notice the conversation takes more rounds to terminate:

In [3]:
import os
from autogen import ConversableAgent

In [4]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
)

In [5]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, I'll give it a try: "Why did the scarecrow win an award? Because he was outstanding in his field!"

This is a play on words, as the word "outstanding" can also refer to being excellent or exceptional, which is why the scarecrow won the award. It's a clever pun that will make you laugh!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Ha ha, that's a great joke!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Thank you! I'm glad you enjoyed it. Do you want to hear another one?

--------------------------------------------------------------------------------


In [6]:
result = joe.initiate_chat(
    cathy, message="Cathy, tell me a joke.", max_turns=3
)  # increase the number of max turns before termination

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, I'll give it a try: "Why did the scarecrow win an award? Because he was outstanding in his field!"

This is a play on words, as the word "outstanding" can also refer to being excellent or exceptional, which is why the scarecrow won the award. It's a clever pun that will make you laugh!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


...

cathy (to joe):

Thank you! I'm glad you enjoyed it. Do you want to hear another one?

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Sure, I'd love to hear it.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, here's another one: "Why did the bicycle fall over? Because it was two-tired!"

This joke is a play on words as well, as the word "tired" can also mean exhausted or worn out. It's a clever pun that will make you laugh again!

--------------------------------------------------------------------------------



## Agent-triggered termination
You can also terminate a conversation by configuring parameters of an agent.
Currently, there are two parameters you can configure:

1. `max_consecutive_auto_reply`: This condition trigger termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the `max_consecutive_auto_reply` argument of the `ConversableAgent` class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention. We will describe this in more detail in the next chapter.
2. `is_termination_msg`: This condition can trigger termination if the _received_ message satisfies a particular condition, e.g., it contains the word "TERMINATE". You can customize this condition using the `is_terminate_msg` argument in the constructor of the `ConversableAgent` class.

### Using `max_consecutive_auto_reply`

In the example below lets set `max_consecutive_auto_reply` to `1` and notice how this ensures that Joe only replies once.

In [7]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1,  # Limit the number of consecutive auto-replies.
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.")

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, I'll give it a try: "Why did the scarecrow win an award? Because he was outstanding in his field!"

This is a play on words, as the word "outstanding" can also refer to being excellent or exceptional, which is why the scarecrow won the award. It's a clever pun that will make you laugh!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Ha ha, that's a great joke!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Thank you! I'm glad you enjoyed it. Do you want to hear another one?

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.


### Using `is_termination_msg`

Let's set the termination message to "GOOD BYE" and see how the conversation terminates.

In [8]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

I'm not sure what you mean by "duo of comedians." Are we both comedians? Or is one of us a comedian and the other a duo? Could you please clarify this for me? I want to make sure that I am being as clear as possible in my response.
 
Additionally, I'm not sure what you mean by "joke." Are you asking for a joke or are you telling me a joke? Again, could you please clarify this for me so that I can provide the best possible response? Thank you.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, let me think of a joke... Okay, here it is: Why was the math book sad? Because it had too many problems! And now, I will say "GOOD BYE" to end our conversation.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.



Notice how the conversation ended based on contents of cathy's message!

## Summary

In this chapter we introduced mechanisms to terminate a conversation between agents.
You can configure both parameters in `initiate_chat` and also configuration of agents.

That said, it is important to note that when a termination condition is triggered,
the conversation may not always terminated immediately. The actual termination
depends on the `human_input_mode` argument of the `ConversableAgent` class.
For example, when mode is `NEVER` the termination conditions above will end the conversations.
But when mode is `ALWAYS` or `TERMINATE`, it will not terminate immediately.
We will describe this behavior and explain why it is important in the next chapter.
